In [9]:
#thresholding

import nibabel as nib
import numpy as np

# Path to the t-contrast NIfTI file
nifti_file_path = r'E:\proje\data\1Heydari\fMRI\MR\Series 7-ep2d_Run5\Run5_nii\1_first_level/spmT_0031.nii'  # Replace with your NIfTI file path

# Load the NIfTI file
img = nib.load(nifti_file_path)
data = img.get_fdata()

# Apply thresholding
thresholded_data = np.where(np.logical_or(data > 2, data < -2), data, 0)

# Create a new NIfTI image using the thresholded data
thresholded_img = nib.Nifti1Image(thresholded_data, img.affine, img.header)

# Define the path to save the thresholded NIfTI file
thresholded_nifti_file_path = r'E:\proje\data\1Heydari\fMRI\MR\mask\build it mask\r5_tresholdspmT_0031.nii'  # Replace with desired path to save

# Save the thresholded NIfTI image
nib.save(thresholded_img, thresholded_nifti_file_path)


In [ ]:
# average with spm imcal

In [10]:
#assymetric left
import nibabel as nib
import numpy as np

# Path to your 79x95x79 NIfTI file
nifti_file_path = r'E:\proje\data\1Heydari\fMRI\MR\mask\build it mask/tresh_average.nii'

# Load the NIfTI file
img = nib.load(nifti_file_path)
data = img.get_fdata()

# Define target dimensions
target_shape = (91, 109, 91)

# Calculate padding along each axis
pad_x = (target_shape[0] - data.shape[0]) 
pad_y = (target_shape[1] - data.shape[1]) 
pad_z = (target_shape[2] - data.shape[2]) 

# Calculate padding for each side (left, right, top, bottom, front, back)
pad_width = (
   (0,12),  # Padding for x-axis (10 voxels on the left, 2 on the right)
    (0,14),   # Padding for y-axis (5 voxels on the left, 3 on the right)
    (0,12)    
)

# Apply symmetric zero-padding
padded_data = np.pad(data, pad_width, mode='constant', constant_values=0)

# Create a new NIfTI image using the padded data
padded_img = nib.Nifti1Image(padded_data, affine=img.affine)

# Save the padded NIfTI image
padded_nifti_file_path = r'E:\proje\data\1Heydari\fMRI\MR\mask\build it mask/lp_tresh_average.nii'
nib.save(padded_img, padded_nifti_file_path)


In [11]:
import numpy as np
import nibabel as nib

# Load t-contrast activity map
t_map_file = r'E:\proje\data\1Heydari\fMRI\MR\mask\build it mask/lp_tresh_average.nii'
t_map_img = nib.load(t_map_file)
t_map_data = t_map_img.get_fdata()

# Load V1 mask
it_mask_path = r'E:\proje\data\0Hemmati\fMRI\MainData\2019_05_22\Image\mask/it_fusiform.nii'
it_mask_img = nib.load(it_mask_path)
it_mask_data = it_mask_img.get_fdata()

# Extract values only within the mask
masked_t_map_values = t_map_data[it_mask_data != 0]

# Get indices of 800 most activated voxels
top_800_indices = np.argsort(masked_t_map_values)[-1200:]

# Initialize new_mask_data with zeros of the same shape as it_mask_data
new_mask_data = np.zeros_like(it_mask_data)

# Get indices in the mask
indices_in_mask = np.where(it_mask_data != 0)

# Create new_mask_data with the 800 most activated voxels set to 1
new_mask_data[indices_in_mask[0][top_800_indices], 
              indices_in_mask[1][top_800_indices], 
              indices_in_mask[2][top_800_indices]] = 1

# Create a new NIfTI image with the header from the original mask
new_mask_img = nib.Nifti1Image(new_mask_data, affine=it_mask_img.affine, header=it_mask_img.header)

# Save the new mask
new_mask_file = r'E:\proje\data\1Heydari\fMRI\MR\mask\build it mask/test_tresh_it_fusiform_heydari.nii'
nib.save(new_mask_img, new_mask_file)

